In [1]:
from numpy import *
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd 

/Users/sivber/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat
class smoSVM:
    def __init__(self,xMatIn,yMatIn,C,toler,kTup,maxIter):
        self.X = xMatIn
        self.Y = yMatIn
        self.C = C
        self.tol = toler
        self.m = shape(xMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        self.K = mat(zeros((self.m,self.m)))
        self.maxIter = maxIter
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
            
    def selectJrand(self,i):
        j=i #we want to select any J not equal to i
        while (j==i):
            j = int(random.uniform(0,self.m))
        return j

    def clipAlpha(self,aj,H,L):
        if aj > H: 
            aj = H
        if L > aj:
            aj = L
        return aj
    def calcEk(self,k):
        fxk = float(multiply(self.alphas,self.Y).T*(self.X*self.X[k,:].T)) + self.b
        Ek = fxk - float(self.Y[k])
        return Ek

    #选取Ei-Ej绝对值最大的j
    def selectJ(self,i,Ei):
        maxK = -1; maxDeltaE = 0; Ej = 0
        #第一列为标志该值是否有效，第二列才为误差值
        self.eCache[i] = [1,Ei]
        #nonzero返回不为0的坐标,[0]为行坐标，[1]为列坐标，[i]为第i纬的坐标
        validEcacheList = nonzero(self.eCache[:,0].A)[0]
        if (len(validEcacheList)) > 1:
            for k in validEcacheList:   
                if k == i: continue 
                Ek = self.calcEk(k)
                deltaE = abs(Ei - Ek)
                if (deltaE > maxDeltaE):
                    maxK = k; maxDeltaE = deltaE; Ej = Ek
            return maxK, Ej
        else:  
            j = self.selectJrand(i)
            Ej = self.calcEk(j)
        return j, Ej
    
    def updateEk(self, k):
        Ek = self.calcEk( k)
        self.eCache[k] = [1,Ek]
        
    def innerL(self,i):
        Ei = self.calcEk(i)
        #KTT : 0 <= alpha <= C toler为容错参数，当超过容错参数同时alpha在约束条件内才继续优化alpha，
        #统计学习方法.李航 p113 支持向量部分
        #Y_train[i]*Ei < -toler表示i被误分类到另外一侧，且在间隔之外，如果alpha<C那么alpha应该增大
        #Y_train[i]*Ei > toler表示i被正分类了，且在间隔之外,如果alpha>0那么alpha应该缩小
        if ((self.Y[i]*Ei<-self.tol) and (self.alphas[i] < oS.C)) or ((oS.Y[i]*Ei>oS.tol) and (self.alphas[i] >0)):
            #计算误差
            j,Ej = self.selectJ(i,Ei)
            alphaIold = self.alphas[i].copy();alphaJold = self.alphas[j].copy();
            #由KTT:sum(alphas*y) = 0
            #当i与j的y同号时相当于alphas[i] + alphas[j] = A(常数) 
            #异号时相当于alphas[i] - alphas[j] = A
            #同时必须满足 0 <= alpha <= C 
            #则分别求出上下界L H 如果上下界相同，则alpha处于边界 直接跳过
            if(self.Y[i]!=self.Y[j]):
                L = max(0,self.Y[j]-self.Y[i])
                H = min(self.C,self.C+self.Y[j]-self.Y[i])
            else:
                L = max(0,-self.C+self.Y[j]+self.Y[i])
                H = min(self.C,self.Y[j]+self.Y[i])
            if L==H: print "L==H"; return 0
            #由sum(y[i]*alpha[i]) = 0 将alpha[j]由alpha[i]表示带入目标函数 可以得到alpha[2]的一元二次方程，求导使其为0可以得到极值点
            #最终alpha[j]=alpha[j]_old + y[j](Ei-Ej)/(K11+K22-2K12) K11 = (X[i].X[j])
            #这里eta为2K12 - K11 - K22
            eta = 2.0*self.K[i,j] - self.K[i,i] - self.K[j,j]
            if eta >= 0: print "eta>=0"; return 0
            self.alphas[j] -= self.Y[j]*(Ei-Ej)/eta
            self.alphas[j] = self.clipAlpha(self.alphas[j],H,L)
            self.updateEk(j)
            if (abs(self.alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; return 0
            self.alphas[i] += self.Y[j]*self.Y[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
            self.updateEk(i) #added this for the Ecache                    #the update is in the oppostie direction
            b1 = self.b - Ei- self.Y[i]*(self.alphas[i]-alphaIold)*self.K[i,i] - self.Y[j]*(self.alphas[j]-alphaJold)*self.K[i,j]
            b2 = self.b - Ej- self.Y[i]*(self.alphas[i]-alphaIold)*self.K[i,j]- self.Y[j]*(self.alphas[j]-alphaJold)*self.K[j,j]
            if (0 < self.alphas[i]) and (self.C > self.alphas[i]): self.b = b1
            elif (0 < self.alphas[j]) and (self.C > self.alphas[j]): self.b = b2
            else: self.b = (b1 + b2)/2.0
            return 1
        else: return 0
    def smoP(self):    #full Platt SMO

        iter = 0
        entireSet = True; alphaPairsChanged = 0
        while (iter < self.maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
            alphaPairsChanged = 0
            if entireSet:   
                for i in range(self.m):        
                    alphaPairsChanged += self.innerL(i)
                    print "fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
                iter += 1
            else:
                nonBoundIs = nonzero((self.alphas.A > 0) * (self.alphas.A < C))[0]
                for i in nonBoundIs:
                    alphaPairsChanged += self.innerL(i)
                    print "non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
                iter += 1
            if entireSet: entireSet = False 
            elif (alphaPairsChanged == 0): entireSet = True  
            print "iteration number: %d" % iter
        return 0